In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 19.6 MB/s eta 0:00:00


In [ ]:
!pip install torch==2.0.1
!pip install torch_geometric==2.3.0 -f https://data.pyg.org/whl/torch-2.0.1+cpu.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_max_pool
from torch_geometric.utils import to_undirected
from sklearn.model_selection import train_test_split
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import is_aa
import matplotlib.pyplot as plt

# Step 1: Define a function to generate 20-dimensional node features based on amino acid properties
def generate_node_features(sequence):
    properties = {
        'G': [-0.4, 0, 75.0],    # Glycine
        'A': [1.8, 0, 89.1],     # Alanine
        'V': [4.2, 0, 117.1],    # Valine
        'L': [3.8, 0, 131.2],    # Leucine
        'I': [4.5, 0, 131.2],    # Isoleucine
        'M': [1.9, 0, 149.2],    # Methionine
        'F': [2.8, 0, 165.2],    # Phenylalanine
        'W': [-3.4, 1, 204.2],   # Tryptophan
        'S': [-0.8, 1, 105.1],   # Serine
        'T': [-0.7, 1, 119.1],   # Threonine
        'C': [2.5, 1, 121.2],    # Cysteine
        'N': [-3.5, 1, 132.1],   # Asparagine
        'Q': [-3.5, 1, 146.2],   # Glutamine
        'D': [-3.5, 1, 133.1],   # Aspartic acid
        'E': [-3.5, 1, 147.1],   # Glutamic acid
        'K': [-3.9, 1, 146.2],   # Lysine
        'R': [-4.5, 1, 174.2],   # Arginine
        'H': [-3.2, 1, 155.2]    # Histidine
    }

    # Map sequence to 20-dimensional feature matrix
    features = [properties.get(aa, [0, 0, 0]) + [0] * (20 - len(properties.get(aa, [0, 0, 0]))) for aa in sequence]
    return np.array(features)

# Step 2: Define the Dataset Class with Updated Node Feature Generation
class ProteinDataset(Dataset):
    def __init__(self, data, adjacency_matrix):
        self.data = data
        self.adjacency_matrix = adjacency_matrix

    def __len__(self):
        return len(self.data)

    def _get_node_features(self, idx):
        sequence = self.data.iloc[idx]['sequence']
        node_features = generate_node_features(sequence)
        return node_features

    def __getitem__(self, idx):
        node_features = self._get_node_features(idx)
        solubility = self.data.iloc[idx]['solubility']

        # Generate edge_index by taking the non-zero entries of the adjacency matrix
        edge_index = torch.tensor(np.array(self.adjacency_matrix.nonzero()), dtype=torch.long)
        edge_index = to_undirected(edge_index)

        # Define node features and target label as tensors
        x = torch.tensor(node_features, dtype=torch.float)
        y = torch.tensor([solubility], dtype=torch.float)

        return Data(x=x, edge_index=edge_index, y=y)

# Step 3: Function to calculate adjacency matrix for residue contacts
def calculate_residue_distance_matrix(residues, threshold=5.0):
    num_residues = len(residues)
    adjacency_matrix = np.zeros((num_residues, num_residues))

    for i, res1 in enumerate(residues):
        for j, res2 in enumerate(residues):
            if i != j:
                min_distance = min(atom1 - atom2 for atom1 in res1 if atom1.element != 'H'
                                                    for atom2 in res2 if atom2.element != 'H')
                if min_distance <= threshold:
                    adjacency_matrix[i, j] = 1  # Contact

    return adjacency_matrix

def pdb_to_adjacency_matrix(file_path, threshold=5.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", file_path)
    residues = [residue for residue in structure.get_residues() if is_aa(residue)]
    adjacency_matrix = calculate_residue_distance_matrix(residues, threshold)
    return adjacency_matrix

# Load Dataset
data = pd.read_csv('/content/eSol_train.csv')  # Update with actual path
file_path = "/content/new_chec_bio.pdb"  # Update with actual PDB file path
adjacency_matrix = pdb_to_adjacency_matrix(file_path, threshold=5.0)
dataset = ProteinDataset(data, adjacency_matrix)
train_data, test_data = train_test_split(dataset, test_size=0.2)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))
test_loader = DataLoader(test_data, batch_size=32, shuffle=True, collate_fn=lambda x: Batch.from_data_list(x))

# Step 4: Define the GCN Model with Global Max Pooling
class GCNModel(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc = torch.nn.Linear(hidden_channels, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = global_max_pool(x, batch)
        return self.fc(x)

# Training and Evaluation Functions
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNModel(num_node_features=20, hidden_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

def train_model(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch)
        loss = loss_fn(out.squeeze(), batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate_model(model, loader, loss_fn):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            out = model(batch)
            loss = loss_fn(out.squeeze(), batch.y)
            total_loss += loss.item()
    return total_loss / len(loader)

# Training Loop
epochs = 50
for epoch in range(1, epochs + 1):
    train_loss = train_model(model, train_loader, optimizer, loss_fn)
    test_loss = evaluate_model(model, test_loader, loss_fn)
    print(f'Epoch {epoch}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

# Final test evaluation
final_test_loss = evaluate_model(model, test_loader, loss_fn)
print(f"Final Test Loss: {final_test_loss:.4f}")

# Save model
model_path = "/content/gcn_model_2.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Function to load the model and make predictions
def load_model(model_path):
    model = GCNModel(num_node_features=20, hidden_channels=16)  # Adjust as per training
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

def make_prediction(file_path, model_path):
    model = load_model(model_path)
    adjacency_matrix, node_features, _ = pdb_to_adjacency_matrix(file_path, threshold=5.0), generate_node_features(sequence)
    edge_index = torch.tensor(np.array(adjacency_matrix.nonzero()), dtype=torch.long)
    edge_index = to_undirected(edge_index)
    x = torch.tensor(node_features, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index)

    with torch.no_grad():
        prediction = model(data)
    return prediction.item()


Epoch 1, Train Loss: 3.3942, Test Loss: 0.1772
Epoch 2, Train Loss: 0.1189, Test Loss: 0.1131
Epoch 3, Train Loss: 0.1112, Test Loss: 0.1127
Epoch 4, Train Loss: 0.1110, Test Loss: 0.1113
Epoch 5, Train Loss: 0.1111, Test Loss: 0.1109
Epoch 6, Train Loss: 0.1090, Test Loss: 0.1127
Epoch 7, Train Loss: 0.1089, Test Loss: 0.1096
Epoch 8, Train Loss: 0.1081, Test Loss: 0.1090
Epoch 9, Train Loss: 0.1098, Test Loss: 0.1088
Epoch 10, Train Loss: 0.1067, Test Loss: 0.1328
Epoch 11, Train Loss: 0.1101, Test Loss: 0.1143
Epoch 12, Train Loss: 0.1084, Test Loss: 0.1092
Epoch 13, Train Loss: 0.1082, Test Loss: 0.1046
Epoch 14, Train Loss: 0.1039, Test Loss: 0.1055
Epoch 15, Train Loss: 0.1051, Test Loss: 0.1096
Epoch 16, Train Loss: 0.1056, Test Loss: 0.1109
Epoch 17, Train Loss: 0.1079, Test Loss: 0.1025
Epoch 18, Train Loss: 0.1077, Test Loss: 0.1042
Epoch 19, Train Loss: 0.1050, Test Loss: 0.1037
Epoch 20, Train Loss: 0.1023, Test Loss: 0.1005
Epoch 21, Train Loss: 0.1057, Test Loss: 0.1118
E

In [ ]:
def extract_sequence_from_pdb(file_path):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", file_path)
    residues = [residue for residue in structure.get_residues() if is_aa(residue)]
    sequence = ''.join([residue.get_resname()[0] for residue in residues])  # First letter for each residue
    return sequence

def make_prediction(file_path, model_path):
    model = load_model(model_path)

    # Extract sequence and adjacency matrix
    sequence = extract_sequence_from_pdb(file_path)
    adjacency_matrix = pdb_to_adjacency_matrix(file_path, threshold=5.0)

    # Generate node features from sequence
    node_features = generate_node_features(sequence)

    # Prepare edge_index
    edge_index = torch.tensor(np.array(adjacency_matrix.nonzero()), dtype=torch.long)
    edge_index = to_undirected(edge_index)

    # Prepare data for model input
    x = torch.tensor(node_features, dtype=torch.float)
    data = Data(x=x, edge_index=edge_index)

    # Make prediction
    with torch.no_grad():
        prediction = model(data)
    return prediction.item()

In [ ]:

user_pdb_file = "/content/new_chec_bio.pdb"  # Replace with actual user input file path
trained_model_path = "/content/gcn_model_2.pth"  # Path to the saved model
prediction = make_prediction(user_pdb_file, trained_model_path)
print(f"Predicted solubility: {prediction:.4f}")

Predicted solubility: 0.4088


In [ ]:

user_pdb_file = "/content/bionemo.pdb"  # Replace with actual user input file path
trained_model_path = "/content/gcn_model_2.pth"  # Path to the saved model
prediction = make_prediction(user_pdb_file, trained_model_path)
print(f"Predicted solubility: {prediction:.4f}")

Predicted solubility: 0.4147


In [ ]:

user_pdb_file = "/content/BIOOOO.pdb"  # Replace with actual user input file path
trained_model_path = "/content/gcn_model_2.pth"  # Path to the saved model
prediction = make_prediction(user_pdb_file, trained_model_path)
print(f"Predicted solubility: {prediction:.4f}")

Predicted solubility: 0.4110


In [ ]:
import requests

invoke_url = "https://health.api.nvidia.com/v1/biology/nvidia/esmfold"

headers = {
    "Authorization": "Bearer Give-your-api-key",
    "Accept": "application/json",
}

payload = {
  "sequence": "MDILCEENTSLSSTTNSLMQLNDDTRLYSNDFNSGEANTSDAFNWTVDSENRTNLSCEGCLSPSCLSLLHLQEKNWSALLTAVVIILTIAGNILVIMAVSLEKKLQNATNYFLMSLAIADMLLGFLVMPVSMLTILYGYRWPLPSKLCAVWIYLDVLFSTASIMHLCAISLDRYVAIQNPIHHSRFNSRTKAFLKIIAVWTISVGISMPIPVFGLQDDSKVFKEGSCLLADDNFVLIGSFVSFFIPLTIMVITYFLTIKSLQKEATLCVSDLGTRAKLASFSFLPQSSLSSEKLFQRSIHREPGSYTGRRTMQSISNEQKACKVLGIVFFLFVVMWCPFFITNIMAVICKESCNEDVIGALLNVFVWIGYLSSAVNPLVYTLFNKTYRSAFSRYIQCQYKENKKPLQLILVNTIPALAYKSSQLQMGQKKNSKQDAKTTDNDCSMVALGKQHSEEASKDNSDGVNEKVSCV"
}

# re-use connections
session = requests.Session()

response = session.post(invoke_url, headers=headers, json=payload)

response.raise_for_status()
response_body = response.json()
print(response_body)


{'pdbs': ['PARENT N/A                                                                      \nMODEL     1                                                                     \nATOM      1  N   MET A   1      11.485 -52.270  -9.149  1.00 68.72           N  \nATOM      2  CA  MET A   1      10.175 -52.100  -8.526  1.00 71.19           C  \nATOM      3  C   MET A   1       9.337 -51.080  -9.290  1.00 67.85           C  \nATOM      4  CB  MET A   1      10.325 -51.667  -7.067  1.00 62.60           C  \nATOM      5  O   MET A   1       9.548 -49.873  -9.157  1.00 62.20           O  \nATOM      6  CG  MET A   1      10.036 -52.774  -6.066  1.00 58.85           C  \nATOM      7  SD  MET A   1      10.350 -52.255  -4.334  1.00 57.93           S  \nATOM      8  CE  MET A   1       9.913 -53.782  -3.458  1.00 57.04           C  \nATOM      9  N   ASP A   2       9.045 -51.337 -10.576  1.00 75.88           N  \nATOM     10  CA  ASP A   2       8.222 -50.972 -11.725  1.00 75.95           C  \nATOM 